# Prompt Tuning

In [1]:
import os
from collections import Counter

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType

import torch
from torch.utils.data import DataLoader

from tqdm import tqdm
import wandb

Make sure to login to wandb first using the cli:
```
wandb login
<paste wandb api key>
```

In [3]:
# initialize project on wandb
wandb.init(project='prompt_learning_methods', name='prompt_tuning')

# set seed for reproducibility
seed = 0

# Hugguingface endpoints
device = 'cuda'
model_name_or_path = "mistralai/Mistral-7B-v0.1"
tokenizer_name_or_path = "mistralai/Mistral-7B-v0.1"
dataset_name = "twitter_complaints"

# Dataset
text_column = "Tweet text"
label_column = "text_label"

# Model hyperparams
max_length = 64
lr = 1e-4
num_epochs = 10
batch_size = 8
set_seed(seed)

Make sure you already accepted access for the repo of this model `mistralai/Mistral-7B-v0.1`

## Dataset prep

In [4]:
dataset = load_dataset("ought/raft", dataset_name)
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)

dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1
)
print(dataset)
dataset["train"][0]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

['Unlabeled', 'complaint', 'no complaint']


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})


{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [5]:
Counter(dataset["train"]["Label"])

Counter({2: 33, 1: 17})

In [6]:
# preprocess dataset
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(f"{target_max_length=}")

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

target_max_length=4


In [7]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x}\nLabel : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
preprocessed_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset"
)

train_dataset = preprocessed_dataset["train"]
eval_dataset = preprocessed_dataset["train"]

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
next(iter(train_dataloader))

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'input_ids': tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     1,   320, 12523,  2245,   714,  3194,
          10198,   264,  8428,  4449,  1508, 28707, 28723,  1115, 28748, 28754,
           1981, 28765, 28727, 28743, 28768,  7209, 28718,    13,  4565,   714,
          28705,   708, 22105,     2],
         [    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     1,   320, 12523,  2245,   714,   802,
          28796, 13773, 28796,  6316, 28790,   315,   776, 24385,   264,  5401,
           3667,  3339,   297,  9231,  3217,  1286,  6247,    13,  4

In [9]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column}: {x}\nLabel" for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    
    return model_inputs

In [10]:
test_dataset = dataset["test"].map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset"
)
test_dataloader = DataLoader(
    test_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
next(iter(test_dataloader))

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'input_ids': tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     1,   320, 12523,  2245, 28747,   802, 28760,
            324, 14233,  2328,  8868,   354, 10313,   586,  7416,  2169,   395,
            272,  4908,  8147,  1309,   356,   378, 28808, 11936, 28723, 22747,
          28723,   675, 28748, 28710, 28737, 28734, 11788, 28744, 28762, 28779,
          28779, 28750,    13,  4565],
         [    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
              2,     2,     2,     2,     2,     2,     2,     2,     1,   320,
          12523,  2245, 28747,   334,   855,   288,   582,   356,   422,  3836,
           3957,  1829, 19653, 28808,   415,   905,   590,  3088,   354,  1167,
           4370,   568,  1371,   272,  8710,   472, 28705, 29137, 29

## Create PEFT model, Optimizer and LR Scheduler

In [11]:
# prompt tunning config
# this is the general instruction
prompt_tunning_init_text = "Classify if the tweet is a complaint or no complaint. \n"
# now peft config
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=len(tokenizer(prompt_tunning_init_text)["input_ids"]),
    prompt_tuning_init_text=prompt_tunning_init_text,
    tokenizer_name_or_path=model_name_or_path
)

In [12]:
# create model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16
)
# now homologate model to peft
model = get_peft_model(model, peft_config)
# print all trainable parameters
model.print_trainable_parameters()
# now enable gradient checkpointing for resume if stopped
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model = model.to(device)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 65,536 || all params: 7,241,797,632 || trainable%: 0.0009


Trainable parameters are 0.0009% of total model parameters!

In [13]:
# define optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.1)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader)*num_epochs)
)

## Qualitative evaluation on test samples before finetuning

In [14]:
model.eval()
i = 33
inputs = tokenizer(
    f'{text_column}: {dataset["test"][i]["Tweet text"]}\nLabel : ',
    return_tensors='pt'
)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=20,
        eos_token_id=tokenizer.eos_token_id
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:1685: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


Tweet text: @TommyHilfiger Dramatic shopping exp. ordered 6 jeans same size (30/32) 2 fits / 2 too large / 2 too slim : same brand &gt; different sizing
Label : 30/32

I’ve been a fan of Tommy Hilfiger for a


Basemodel does not perform as expected, cause in example `label: 30/32` does not seem to represent an intuitive label at all.

## Training and evaluation loop

In [15]:
# training and evaluation
for epoch in range(num_epochs):
    # state model on train stage
    model.train()
    # this will count loss over each epoch
    total_loss = 0
    # start iterating over each batch on train dataloader
    for step, batch in enumerate(tqdm(train_dataloader)):
        # send batch values to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.autocast(dtype=torch.float16, device_type="cuda"):
            # map batch values to model and get outputs on forward pass
            outputs = model(**batch)
        # get loss from output
        loss = outputs.loss
        # start accumulating losses over each epoch
        total_loss += loss.detach().float()
        # do backward step
        loss.backward()
        # optimize on backward step
        optimizer.step()
        # define if learning rate must be redefine on this step
        lr_scheduler.step()
        # reset gradients for next step
        optimizer.zero_grad()

    # state model on evaluation stage
    model.eval()
    # this will count loss over each epoch for evaluation set
    eval_loss = 0
    eval_preds = []
    # start iterating over each batch on evaluation dataloader
    for step, batch in enumerate(tqdm(eval_dataloader)):
        # send batch values to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            # use no gradients for this cause this is eval only
            # map batch values to model and get outputs
            outputs = model(**batch)
        # get loss from evaluation outputs
        loss = outputs.loss
        # start accumulating losses over each epoch
        eval_loss += loss.detach().float()
        # storage predictions
        eval_preds.extend(
            tokenizer.batch_decode(
                torch.argmax(outputs.logits, -1).detach().cpu().numpy(),
                skip_special_tokens=True
            )
        )

    # get average of eval losses
    eval_epoch_loss = eval_loss / len(eval_dataloader)
    # get perpelexity of eval losses
    # perplexity: measure of how well a model is accomplishing its task.
    # read further info here: https://developers.google.com/machine-learning/glossary#perplexity
    eval_ppl = torch.exp(eval_epoch_loss)
    # get average of train losses
    train_epoch_loss = total_loss / len(train_dataloader)
    # get perpelexity of train losses
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

    # log metrics on wandb
    wandb.log(
        {
            "train": {"perplexity": train_ppl, "loss": train_epoch_loss, "epoch": epoch},
            "val": {"perplexity": eval_ppl, "loss": eval_epoch_loss, "epoch": epoch} 
        }
    )

100%|██████████| 7/7 [00:00<00:00,  9.05it/s]


epoch=0: train_ppl=tensor(31.5284, device='cuda:0') train_epoch_loss=tensor(3.4509, device='cuda:0') eval_ppl=tensor(8.2772, device='cuda:0') eval_epoch_loss=tensor(2.1135, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00,  8.98it/s]


epoch=1: train_ppl=tensor(4.7345, device='cuda:0') train_epoch_loss=tensor(1.5549, device='cuda:0') eval_ppl=tensor(2.8114, device='cuda:0') eval_epoch_loss=tensor(1.0337, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00,  9.00it/s]


...

epoch=7: train_ppl=tensor(1.0732, device='cuda:0') train_epoch_loss=tensor(0.0706, device='cuda:0') eval_ppl=tensor(1.0548, device='cuda:0') eval_epoch_loss=tensor(0.0534, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00,  9.10it/s]


epoch=8: train_ppl=tensor(1.0502, device='cuda:0') train_epoch_loss=tensor(0.0490, device='cuda:0') eval_ppl=tensor(1.0473, device='cuda:0') eval_epoch_loss=tensor(0.0462, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00,  9.07it/s]

epoch=9: train_ppl=tensor(1.0450, device='cuda:0') train_epoch_loss=tensor(0.0440, device='cuda:0') eval_ppl=tensor(1.0449, device='cuda:0') eval_epoch_loss=tensor(0.0439, device='cuda:0')


## Qualitative evaluation on test samples after finetuning

In [20]:
model.eval()
i = 33
inputs = tokenizer(
    f'{text_column}: {dataset["test"][i]["Tweet text"]}\nLabel: ',
    return_tensors="pt"
)
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=target_max_length+1,
        eos_token_id=tokenizer.eos_token_id
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:1685: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


<s> Tweet text: @TommyHilfiger Dramatic shopping exp. ordered 6 jeans same size (30/32) 2 fits / 2 too large / 2 too slim : same brand &gt; different sizing
Label: 2015-


Well label says 2015 which is not a label class of the defined in the dataset :(

The guide pushes the model to huggingface hub and then load it again to evaluate on test samples. </br>
```python
model.push_to_hub(
    f"mistral_prompt_tuning",
    token="hf_...",
    private=True
)
```

Or save locally:
```python
peft_model_id = "mistral_prompt_tuning",
model.save_pretrained(peft_model_id)
```

In [17]:
classes

['Unlabeled', 'complaint', 'no complaint']

**In any case, the model checkpoint is extremelly much smaller in size than original model, which improves a lot the deployment of this model in downstream tasks.**